## Week 2 Day 3

Now we get to more detail with the Google Gemini API:

1. Structured Outputs
2. Guardrails

In [ ]:
from dotenv import load_dotenv
from google import genai
from google.genai import types
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from pydantic import BaseModel

In [ ]:
load_dotenv(override=True)

In [ ]:
client = genai.Client()

In [ ]:
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    try:
        sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
        from_email = Email("test@example.com")  # Change to your verified sender
        to_email = To("test@example.com")  # Change to your recipient
        content = Content("text/html", html_body)
        mail = Mail(from_email, to_email, subject, content).get()
        sg.client.mail.send.post(request_body=mail)
        return {"status": "success"}
    except Exception as e:
        return {"status": "error", "message": str(e)}

def write_email_subject(email_body: str) -> str:
    """Given the body of an email, write a compelling subject line."""
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[f"Write a subject for this email:\n\n{email_body}"],
        config=types.GenerateContentConfig(system_instruction="You write compelling email subject lines.")
    )
    return response.text

def convert_to_html(email_body: str) -> str:
    """Given a plain text email, convert it to a simple, well-formatted HTML email."""
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[f"Convert this to HTML:\n\n{email_body}"],
        config=types.GenerateContentConfig(system_instruction="You convert plain text to simple, elegant HTML.")
    )
    return response.text

## Part 1: Structured Outputs

We can force the model to return a response that conforms to a specific Pydantic schema.

In [ ]:
class NameCheckOutput(BaseModel):
    """Pydantic schema for checking if a name is in the message."""
    is_name_in_message: bool
    name: str

In [ ]:
def check_for_name(message: str) -> NameCheckOutput:
    """Checks if a personal name is included in the user's message using structured output."""
    response = client.models.generate_content(
        model='gemini-2.5-flash',
        contents=[message],
        config=types.GenerateContentConfig(
            system_instruction="Check if the user is including someone's personal name in what they want you to do.",
            response_mime_type='application/json',
            response_schema=NameCheckOutput,
        ),
    )
    # The response.text is a JSON string that can be parsed into our Pydantic model
    return NameCheckOutput.model_validate_json(response.text)

## Part 2: Guardrails

We can use our structured output function as a 'guardrail' to check input before proceeding.

In [ ]:
async def careful_sales_manager(message: str):
    """An orchestrated workflow with a guardrail to check for names before sending an email."""
    
    # 1. Guardrail: Check for a personal name in the message
    print("Checking for personal names...")
    name_check_result = check_for_name(message)
    
    if name_check_result.is_name_in_message:
        print(f"Guardrail triggered! Found name: {name_check_result.name}. Aborting.")
        return
    else:
        print("No personal names found. Proceeding.")

    # 2. If guardrail is passed, proceed with the sales workflow
    sales_manager_instructions = """
    You are a Sales Manager at ComplAI. Your goal is to find the single best cold sales email.
    Follow these steps carefully:
    1. Generate a professional email draft.
    2. Generate an engaging email draft.
    3. Generate a concise email draft.
    4. Review the drafts and choose the single best one.
    5. Once you have the best email body, use the provided tools to get a subject, convert it to HTML, and send it.
    """
    
    tools = [write_email_subject, convert_to_html, send_html_email]
    
    # This is a simplified orchestration for demonstration.
    # A more robust solution would use a loop to manage the multi-step tool use.
    print("\nGenerating email drafts...")
    professional_email = client.models.generate_content('gemini-2.5-flash', [message], config=types.GenerateContentConfig(system_instruction="You write professional, serious cold emails.")).text
    engaging_email = client.models.generate_content('gemini-2.5-flash', [message], config=types.GenerateContentConfig(system_instruction="You write witty, engaging cold emails.")).text
    concise_email = client.models.generate_content('gemini-2.5-flash', [message], config=types.GenerateContentConfig(system_instruction="You write concise, to the point cold emails.")).text
    
    print("Selecting the best email...")
    selection_prompt = f"Choose the best email from these three options:\n\n1. {professional_email}\n\n2. {engaging_email}\n\n3. {concise_email}"
    best_email_body = client.models.generate_content('gemini-2.5-flash', [selection_prompt], config=types.GenerateContentConfig(system_instruction="Select the best email and return only its text.")).text
    
    print(f"\nBest email body:\n{best_email_body}")
    
    print("\nFormatting and sending...")
    subject = write_email_subject(best_email_body)
    html_body = convert_to_html(best_email_body)
    send_html_email(subject, html_body)
    
    print("\nEmail sent successfully!")

In [ ]:
message_with_name = "Send out a cold sales email addressed to Dear CEO from Alice"
await careful_sales_manager(message_with_name)

In [ ]:
message_without_name = "Send out a cold sales email addressed to Dear CEO from Head of Business Development"
await careful_sales_manager(message_without_name)

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercise</h2>
            <span style="color:#ff7800;">• Add more input and output guardrails (e.g., check for profanity, ensure email is positive in sentiment).<br/>• Use structured outputs for the email generation itself to get back a subject, body, and tone analysis.
            </span>
        </td>
    </tr>
</table>